In [76]:
import pandas as pd    
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [77]:
# Launch web driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [78]:
# Load google maps url
url = 'https://www.google.com/maps/place/CarTimes+Autolution+@+SinMing/@1.3595828,103.8361185,17z/data=!3m2!4b1!5s0x31da172e4e7d5c0b:0xcdc65f29ab85b4a5!4m6!3m5!1s0x31da17c644cef3a1:0x187163161ac34209!8m2!3d1.3595774!4d103.8386934!16s%2Fg%2F11ght0mjh_?entry=ttu'
# https://www.google.com/maps/search/?api=1&query=Google&query_place_id=ChIJIS56-XFmZIgRsvTzijiFReo
driver.get(url)

# Click reviews link
driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]').click()
time.sleep(5)


# # Click sort button
# wait = WebDriverWait(driver, 10)
# menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-value=\'Sort\']')))  
# menu_bt.click()

# # Sort by newest reviews - got issue!!!!!!!
# recent_rating_bt = driver.find_elements('xpath', '//li[@data-index=\'1\']')[0]
# recent_rating_bt.click()
# time.sleep(5)

In [79]:
# Parse html page
response = BeautifulSoup(driver.page_source, 'html.parser')

# Find total number of reviews -  got issue!!!!!!!!
# total_reviews = int(response.find('div', class_ = 'fontBodySmall').text.replace(' reviews', ''))
total_reviews=76

# Find scroll layout
scrollable_div = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll as many times as necessary to load all reviews
for i in range(0, (round(total_reviews / 5 - 1))):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(1)

In [92]:
xpath = '//div[@class="fontBodySmall"]'
reviews_element = soup.find('div', attrs={'/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[3]': xpath})
print(reviews_element)

None


In [84]:
# Re-parse html page after loading all reviews
response = BeautifulSoup(driver.page_source, 'html.parser')
print(response)

<html itemscope="" itemtype="http://schema.org/Place" jstcache="0" lang="en-SG" style="--noto-ff: 'Noto Sans Tamil UI';"><head> <link as="script" href="/maps/_/js/k=maps.m.en.FfwhN74HhDw.2019.O/m=sc2,per,mo,lp,ep,ti,ds,stx,dwi,enr,pwd,dw,ppl,log,std,b/am=JgAjAAI/rt=j/d=1/rs=ACT90oFE4nNtEufXU1kQQJQmPSWfKZu6Rw?wli=m.PCMG9OchpFY.loadSv.O%3A%3Bm.tKdTpQsp2-U.mapcore.O%3A%3B" nonce="" rel="preload" type="application/javascript"/> <link href="/maps/preview/opensearch.xml?hl=en" rel="search" title="Google Maps" type="application/opensearchdescription+xml"/> <title>CarTimes Autolution @ SinMing - Google Maps</title> <meta content="Find local businesses, view maps and get driving directions in Google Maps." name="Description"/> <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-

In [85]:
# Extract reviews
review_ids = [review_id['data-review-id'] for review_id in response.findAll('div', class_= 'jftiEf')]
review_dates = [review_date.text for review_date in response.findAll('span', class_ = 'rsqaWe')]
reviewers = [reviewer['aria-label'] for reviewer in response.findAll('div', class_= 'jftiEf')]
ratings = [rating['aria-label'].strip() for rating in response.findAll('span', class_ = 'kvMYJc')]
review_texts = [review_text.text for review_text in response.findAll('span', class_ = 'wiI7pd')]

In [86]:
# Using len() function
length_using_len = len(review_ids)
print("review_ids:", length_using_len)

length_using_len = len(review_dates)
print("review_dates:", length_using_len)

length_using_len = len(reviewers)
print("reviewers:", length_using_len)

length_using_len = len(ratings)
print("ratings:", length_using_len)

length_using_len = len(review_texts)
print("review_texts:", length_using_len)

review_ids: 76
review_dates: 76
reviewers: 76
ratings: 76
review_texts: 59


In [87]:
combined_list = []

for rating, review_text in zip(ratings, review_texts):
    combined_item = f"Rating: {rating}, Review Text: {review_text}"
    combined_list.append(combined_item)
    
reviews = pd.DataFrame({'Rating': [item.split(', ')[0].split(': ')[1] for item in combined_list],
                   'Review': [item.split(', ')[1].split(': ')[1] for item in combined_list]})

reviews

,Rating,Review
0,5 stars,I have brought our cars here for many years an...
1,5 stars,Was served by Tony and he was very attentive t...
2,5 stars,Nice workshop and friendly staff. First time t...
3,5 stars,Well done Cartimes Autolution. My servicing wa...
4,5 stars,Impressed at the high level of service provide...
5,5 stars,Swift service
6,4 stars,Had some issues with our vehicle & was looking...
7,5 stars,Good customer services with affordable pricing...
8,5 stars,One of the best car servicing which I ever com...
9,5 stars,Fast excellent service centre. Friendly staff ...


In [88]:
# Assume you already have a DataFrame named review_texts
company = 'cartimes'
file_path = f'./{company}_reviews.csv'  # Use './' to save in the current directory

# Saving the DataFrame to a CSV file with the specified path
df.to_csv(file_path, index=False) 
